This is a follow-along notebook of [Agents in LlamaIndex](https://colab.research.google.com/#fileId=https%3A//huggingface.co/agents-course/notebooks/blob/main/unit2/llama-index/agents.ipynb) from [Hugging Face Agents Course](https://huggingface.co/learn/agents-course/unit2/llama-index/agents), with additional trials. 

# Agents in LlamaIndex
> *An Agent is a system that leverages an AI model to interact with its environment to achieve a user-defined objective. It combines reasoning, planning, and action execution (often via external tools) to fulfil tasks.*

LlamaIndex supports three main types of reasoning agents:
1.  `Function Calling Agents` - These work with AI models that can call specific functions.
2.  `ReAct Agents` - These can work with any AI that does chat or text endpoint and deal with complex reasoning tasks.
3.  `Advanced Custom Agents` - These use more complex methods to deal with more complex tasks and workflows, like LLMCompiler or Chain-of-Abstraction. 

## Install Dependencies

In [3]:
!pip install llama-index llama-index-vector-stores-chroma llama-index-llms-huggingface-api llama-index-embeddings-huggingface -Uqq

## Initialize Agents
Let's start by initialising an agent. We will use the basic `AgentWorkflow` class to create an agent.

In [4]:
from llama_index.llms.ollama import Ollama
from llama_index.core.agent.workflow import AgentWorkflow, ToolCallResult, AgentStream

In [5]:
# Create calculation functions
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two numbers"""
    return a - b


def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


def divide(a: int, b: int) -> int:
    """Divide two numbers"""
    return a / b

In [6]:
# Initialize model
llm = Ollama(model="qwen3:8b", request_timeout=60.0)

In [7]:
# Initialize agent
agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[subtract, multiply, divide, add], 
    llm=llm, 
    system_prompt="You are a math agent that can add, subtract, multiply, and divide numbers using provided tools.",
)

In [8]:
# Run agent and print out tool call and thought processes
handler = agent.run("What is (2 + 2) * 2?")
async for ev in handler.stream_events(): 
    if isinstance(ev, ToolCallResult): 
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)

    elif isinstance(ev, AgentStream):  # show thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp


Called tool:  add {'a': 2, 'b': 2} => 4

Called tool:  multiply {'a': 4, 'b': 2} => 8
<think>
Okay, let's see. The user asked for (2 + 2) * 2. First, I need to handle the addition inside the parentheses. So I called the add function with a=2 and b=2, which gave 4. Then, I multiplied that result by 2 using the multiply function. The final answer is 8. I should present that clearly.
</think>

The result of (2 + 2) * 2 is **8**.

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text="<think>\nOkay, let's see. The user asked for (2 + 2) * 2. First, I need to handle the addition inside the parentheses. So I called the add function with a=2 and b=2, which gave 4. Then, I multiplied that result by 2 using the multiply function. The final answer is 8. I should present that clearly.\n</think>\n\nThe result of (2 + 2) * 2 is **8**.")]), tool_calls=[ToolCallResult(tool_name='add', tool_kwargs={'a': 2, 'b': 2}, tool_id='add', tool_output=ToolOutput(content='4', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 2, 'b': 2}}, raw_output=4, is_error=False), return_direct=False), ToolCallResult(tool_name='multiply', tool_kwargs={'a': 4, 'b': 2}, tool_id='multiply', tool_output=ToolOutput(content='8', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 4, 'b': 2}}, raw_output=8, is_error=False), return_direct=F

In [9]:
ev

StopEvent()

In [10]:
handler

<WorkflowHandler finished result=AgentOutput(r..._name='Agent')>

In a similar fashion, we can pass state and context to the agent.

In [11]:
from llama_index.core.workflow import Context

In [ ]:
# Set up context (agent memory = past chat history)
ctx = Context(agent)

In [14]:
ctx

In [15]:
# Add `Context` to run agent
response = await agent.run("My name is Karen", ctx=ctx)
response = await agent.run("What was my name again?", ctx=ctx)
response

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='<think>\nOkay, the user is asking, "What was my name again?" Let me check the conversation history.\n\nLooking back, the user first introduced themselves as Karen. The assistant responded with "Hello, Karen! How can I assist you today?" Then the user asked, "What was my name again?" which seems like they might be confused or testing if I remember.\n\nSince the assistant is supposed to remember the user\'s name from the initial greeting, the answer should be Karen. There\'s no need to use any of the math functions here because the question is about recalling the user\'s name, not performing a calculation. The tools provided (add, subtract, multiply, divide) aren\'t relevant to this query. So, the correct response is to simply state the user\'s name again.\n</think>\n\nYour name is Karen. 😊')]), tool_calls=[], raw={'model': 'qwen3:8b

In [16]:
print(response)

<think>
Okay, the user is asking, "What was my name again?" Let me check the conversation history.

Looking back, the user first introduced themselves as Karen. The assistant responded with "Hello, Karen! How can I assist you today?" Then the user asked, "What was my name again?" which seems like they might be confused or testing if I remember.

Since the assistant is supposed to remember the user's name from the initial greeting, the answer should be Karen. There's no need to use any of the math functions here because the question is about recalling the user's name, not performing a calculation. The tools provided (add, subtract, multiply, divide) aren't relevant to this query. So, the correct response is to simply state the user's name again.
</think>

Your name is Karen. 😊


In [18]:
ctx.is_running

False

In [21]:
ctx.to_dict()

{'globals': {'memory': '{"__is_component": true, "value": {"chat_store": {"store": {"chat_history": [{"role": "user", "additional_kwargs": {}, "blocks": [{"block_type": "text", "text": "My name is Karen"}]}, {"role": "assistant", "additional_kwargs": {"tool_calls": []}, "blocks": [{"block_type": "text", "text": "<think>\\nOkay, the user said, \\"My name is Karen.\\" Let me think about how to respond. Since the user introduced themselves, I should greet them politely. The tools provided are for math operations, but this isn\'t a math question. So I don\'t need to use any of the functions here. I\'ll just reply with a friendly welcome message. Something like, \\"Hello, Karen! How can I assist you today?\\" That should cover it without any function calls.\\n</think>\\n\\nHello, Karen! How can I assist you today?"}]}, {"role": "user", "additional_kwargs": {}, "blocks": [{"block_type": "text", "text": "What was my name again?"}]}, {"role": "assistant", "additional_kwargs": {"tool_calls": []

In [ ]:
# Create tool functions
def bmi_status(weight: float, height: float) -> str:
    """
    This tool caculates the Body Mass Index (BMI) based on weight in kilograms and height in meters.
    It returns a BMI category (Underweight, Normal, Overweight, or Obese).
    """
    bmi = weight / (height ** 2)

    if bmi <= 18.5:
        return "Underweight"
    elif bmi <= 25.0:
        return "Normal"
    elif bmi <= 30.0:
        return "Overweight"
    else:
        return "Obese"


In [ ]:
# Synthetic patient data
data = [
    {"name": "Alice", "weight": 50, "height": 1.65},
    {"name": "Bob", "weight": 85, "height": 1.75},
    {"name": "Charlie", "weight": 70, "height": 1.80},
    {"name": "Diana", "weight": 45, "height": 1.60},
    {"name": "Ethan", "weight": 100, "height": 1.70},
    {"name": "Fiona", "weight": 60, "height": 1.72},
    {"name": "George", "weight": 120, "height": 1.85},
    {"name": "Hannah", "weight": 55, "height": 1.68},
    {"name": "Ian", "weight": 90, "height": 1.78},
    {"name": "Julia", "weight": 40, "height": 1.55}
]

In [48]:
# Initialize agent
agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[bmi_status], 
    llm=llm, 
    system_prompt="You are a clinic assistant analysing patient data. You help calculate BMI and return health status using provided tools."
)

In [29]:
agent.get_prompts()

{'handoff_prompt': PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['agent_info'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template="Useful for handing off to another agent.\nIf you are currently not equipped to handle the user's request, or another agent is better suited to handle the request, please hand off to the appropriate agent.\n\nCurrently available agents:\n{agent_info}\n"),
 'handoff_output_prompt': PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['to_agent', 'reason'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Agent {to_agent} is now handling the request due to the following reason: {reason}.\nPlease continue with the current request.'),
 'state_prompt': PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['state', 'msg'], kwargs={}, output_parser=None, template_var_mapping

In [31]:
agent.state_prompt

PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['state', 'msg'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Current state:\n{state}\n\nCurrent message:\n{msg}\n')

In [32]:
agent.handoff_output_prompt

PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['to_agent', 'reason'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Agent {to_agent} is now handling the request due to the following reason: {reason}.\nPlease continue with the current request.')

In [41]:
agent.get_tools

<bound method AgentWorkflow.get_tools of <llama_index.core.agent.workflow.multi_agent_workflow.AgentWorkflow object at 0x15d98fad0>>

In [ ]:
# Adjust LLM timeout
# DEFAULT_REQUEST_TIMEOUT = 30.0
llm = Ollama(model="qwen3:8b", request_timeout=600.0)

# Initialize agent
agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[bmi_status], 
    llm=llm, 
    system_prompt="You are a clinic assistant analysing patient data. You help calculate BMI and return health status using provided tools."
)

In [49]:
# Run agent
handler = agent.run(f"""
    Patient Data: {data}
    Calculate bmi and return status for these patients, return result in json format with name, bmi and health status.  
""")
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing the thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp


Called tool:  bmi_status {'height': 1.65, 'weight': 50} => Underweight

Called tool:  bmi_status {'height': 1.75, 'weight': 85} => Overweight

Called tool:  bmi_status {'height': 1.8, 'weight': 70} => Normal

Called tool:  bmi_status {'height': 1.6, 'weight': 45} => Underweight

Called tool:  bmi_status {'height': 1.7, 'weight': 100} => Obese

Called tool:  bmi_status {'height': 1.72, 'weight': 60} => Normal

Called tool:  bmi_status {'height': 1.85, 'weight': 120} => Obese

Called tool:  bmi_status {'height': 1.68, 'weight': 55} => Normal

Called tool:  bmi_status {'height': 1.78, 'weight': 90} => Overweight

Called tool:  bmi_status {'height': 1.55, 'weight': 40} => Underweight
<think>
Okay, I need to process the patient data and calculate their BMI status. Let me start by recalling the BMI categories. BMI is calculated as weight divided by height squared. The categories are Underweight (BMI < 18.5), Normal (18.5–24.9), Overweight (25–29.9), and Obese (≥30).

Looking at the patients

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='<think>\nOkay, I need to process the patient data and calculate their BMI status. Let me start by recalling the BMI categories. BMI is calculated as weight divided by height squared. The categories are Underweight (BMI < 18.5), Normal (18.5–24.9), Overweight (25–29.9), and Obese (≥30).\n\nLooking at the patients:\n\n1. **Alice**: Weight 50 kg, height 1.65 m. Let me calculate BMI: 50/(1.65^2) ≈ 50/2.72 ≈ 18.4. So Underweight.\n2. **Bob**: 85 kg, 1.75 m. 85/(1.75^2) ≈ 85/3.06 ≈ 27.7. Overweight.\n3. **Charlie**: 70 kg, 1.8 m. 70/(1.8^2) ≈ 70/3.24 ≈ 21.6. Normal.\n4. **Diana**: 45 kg, 1.6 m. 45/(1.6^2) = 45/2.56 ≈ 17.6. Underweight.\n5. **Ethan**: 100 kg, 1.7 m. 100/(1.7^2) ≈ 100/2.89 ≈ 34.6. Obese.\n6. **Fiona**: 60 kg, 1.72 m. 60/(1.72^2) ≈ 60/2.96 ≈ 20.3. Normal.\n7. **George**: 120 kg, 1.85 m. 120/(1.85^2) ≈ 120/3.42 ≈ 35.1. Obese

In [50]:
# Run agent
handler = agent.run(f"""
    Patient Data: {data}
    Calculate bmi and return status for these patients, return as a pandas dataframe with name, bmi and health status. 
""")
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing the thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp


Called tool:  bmi_status {'height': 1.65, 'weight': 50} => Underweight

Called tool:  bmi_status {'height': 1.75, 'weight': 85} => Overweight

Called tool:  bmi_status {'height': 1.8, 'weight': 70} => Normal

Called tool:  bmi_status {'height': 1.6, 'weight': 45} => Underweight

Called tool:  bmi_status {'height': 1.7, 'weight': 100} => Obese

Called tool:  bmi_status {'height': 1.72, 'weight': 60} => Normal

Called tool:  bmi_status {'height': 1.85, 'weight': 120} => Obese

Called tool:  bmi_status {'height': 1.68, 'weight': 55} => Normal

Called tool:  bmi_status {'height': 1.78, 'weight': 90} => Overweight

Called tool:  bmi_status {'height': 1.55, 'weight': 40} => Underweight
<think>
Okay, let me start by understanding the user's request. They provided a list of patient data with names, weights, and heights. The task is to calculate the BMI for each patient and determine their health status using the bmi_status function. Then, compile the results into a pandas DataFrame with colum

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text="<think>\nOkay, let me start by understanding the user's request. They provided a list of patient data with names, weights, and heights. The task is to calculate the BMI for each patient and determine their health status using the bmi_status function. Then, compile the results into a pandas DataFrame with columns for name, BMI, and health status.\n\nFirst, I need to recall how BMI is calculated. BMI is weight (kg) divided by height (m) squared. The function provided, bmi_status, takes weight and height as parameters and returns the category. The categories are Underweight, Normal, Overweight, or Obese. \n\nLooking at the patient data, each entry has a name, weight, and height. For each patient, I'll need to call the bmi_status function with their specific weight and height. The user already provided the function calls and the respon

## Create RAG Agents with `QueryEngineTool`

In [ ]:
import chromadb

from llama_index.core import VectorStoreIndex
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.tools import QueryEngineTool
from llama_index.vector_stores.chroma import ChromaVectorStore

# Create vector store
db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection("alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Create query engine
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
llm = Ollama(model="qwen3:8b", request_timeout=60.0)
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embed_model
)
query_engine = index.as_query_engine(llm=llm)
query_engine_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="personas",
    description="descriptions for various types of personas",
    return_direct=False,
)

# Create RAG agent
query_engine_agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[query_engine_tool], 
    llm=llm, 
    system_prompt="You are a helpful assistant that has access to a database containing persona descriptions. ",
)

In [51]:
# Run agent
handler = query_engine_agent.run("Search the database for dance educator and return some persona descriptions.")
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp


Called tool:  personas {'input': 'dance educator'} => <think>
Okay, the user is asking about a "dance educator" in the context of the provided information. Let me check the context again. The context mentions a social justice educator or activist focused on diversity, equity, and inclusion, working with families and communities to promote empathy and understanding of intersectional identity and oppression. The file path is repeated twice, but the content is the same.

Now, the query is "dance educator." The answer needs to be based solely on the context given. The context doesn't mention dance specifically. It talks about a social justice educator, which could be in any field, like education, activism, etc. But there's no direct link to dance. However, maybe the user is implying that the dance educator is part of the social justice work mentioned. But the context doesn't specify that. Since the answer must not reference the context directly, I need to infer based on the given info. Th

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='<think>\n</think>\n\nThe persona described is a social justice educator or activist focused on promoting diversity, equity, and inclusion through work with families and communities. While the role involves fostering empathy and understanding of intersectional identity and oppression, there is no specific mention of dance education or related activities in the provided context.')]), tool_calls=[ToolCallResult(tool_name='personas', tool_kwargs={'input': 'dance educator'}, tool_id='personas', tool_output=ToolOutput(content='<think>\nOkay, the user is asking about a "dance educator" in the context of the provided information. Let me check the context again. The context mentions a social justice educator or activist focused on diversity, equity, and inclusion, working with families and communities to promote empathy and understanding of

In [52]:
# Run agent
handler = query_engine_agent.run("Search the database for law professor and return some persona descriptions.")
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp


Called tool:  personas {'input': 'law professor'} => <think>
Okay, the user asked about a "law professor" and provided some context files. Let me check the context info. The files mention an environmental conservationist focused on wetland ecosystems and climate change mitigation. There's no mention of a law professor or any legal aspects. The answer should be based solely on the context given. Since the context doesn't include anything about a law professor, I need to state that there's no relevant information provided. I should avoid referencing the context directly and not mention the files. Just a straightforward answer that the information isn't available in the given context.
</think>

The provided context information does not include any details related to a law professor.
<think>
</think>

The provided context information does not include any details related to a law professor.

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='<think>\n</think>\n\nThe provided context information does not include any details related to a law professor.')]), tool_calls=[ToolCallResult(tool_name='personas', tool_kwargs={'input': 'law professor'}, tool_id='personas', tool_output=ToolOutput(content='<think>\nOkay, the user asked about a "law professor" and provided some context files. Let me check the context info. The files mention an environmental conservationist focused on wetland ecosystems and climate change mitigation. There\'s no mention of a law professor or any legal aspects. The answer should be based solely on the context given. Since the context doesn\'t include anything about a law professor, I need to state that there\'s no relevant information provided. I should avoid referencing the context directly and not mention the files. Just a straightforward answer tha

## Create Multi-Agent Systems
We can also create multi-agent systems by passing multiple agents to the `AgentWorkflow` class.

In [53]:
from llama_index.core.agent.workflow import AgentWorkflow, ReActAgent

In [54]:
# Define tools
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b

In [ ]:
# Create agent configs
# NOTE: We can use `FunctionAgent` or `ReActAgent` here. 
# `FunctionAgent` works for LLMs with a function calling API. 
# `ReActAgent` works for any LLMs. 

# Adjust LLM timeout
llm = Ollama(model="qwen3:8b", request_timeout=600.0)

calculator_agent = ReActAgent(
    name="calculator", 
    description="Performs basic arithmetic operations", 
    system_prompt="You have a calculator assistant. Use your tools for any math operation.", 
    tools=[add, subtract], 
    llm=llm
)

query_agent = ReActAgent(
    name="info_lookup", 
    description="Looks up information about persona descriptions", 
    system_prompt="Use your tool to query a RAG system to answer information about specific persona.", 
    tools=[query_engine_tool], 
    llm=llm
)

In [ ]:
# Create and run the workflow
agent = AgentWorkflow(
    agents=[calculator_agent, query_agent], 
    root_agent="calculator"
)

# Set up handler
handler = agent.run(user_msg="Search the database for law professor and return some persona descriptions, then calculate 7 + 3 - 2")

In [60]:
# Run the system
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp

<think>
Okay, let's tackle this step by step. The user wants two things: first, to search a database for law professor persona descriptions, and then calculate 7 + 3 - 2.

Looking at the tools available, there's the 'handoff' tool which can direct to another agent. The 'info_lookup' agent is mentioned, which is for looking up persona descriptions. So I should use the handoff tool to send the search request to the info_lookup agent. The reason would be to get the persona descriptions as part of the task.

Once that's done, the calculation part is straightforward. The user then wants to subtract 2 from the sum of 7 and 3. Using the add tool first to add 7 and 3, which gives 10, then subtract 2 using the subtract tool, resulting in 8. But wait, the user wrote 7 + 3 - 2, which can be done in one step. Maybe do 7 + 3 first, then subtract 2. So first action is add with 7 and 3, then subtract the result by 2. 

But I need to make sure the handoff is done first. Wait, the user's request has tw

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='8')]), tool_calls=[ToolCallResult(tool_name='handoff', tool_kwargs={'to_agent': 'info_lookup', 'reason': 'Search the database for law professor persona descriptions'}, tool_id='c99e7ef9-15d3-4743-82a1-6e49b9d24ed7', tool_output=ToolOutput(content='Agent info_lookup is now handling the request due to the following reason: Search the database for law professor persona descriptions.\nPlease continue with the current request.', tool_name='handoff', raw_input={'args': (), 'kwargs': {'to_agent': 'info_lookup', 'reason': 'Search the database for law professor persona descriptions'}}, raw_output='Agent info_lookup is now handling the request due to the following reason: Search the database for law professor persona descriptions.\nPlease continue with the current request.', is_error=False), return_direct=True), ToolCallResult(tool_name='per